In [27]:
import sys
sys.path.append('../scripts/')
from utility import read_file, create_folder, temp_record_query, temp_record_sdf

In [28]:
from pyspark.sql import SparkSession

In [29]:
spark = (
    # Create a spark session (which will run spark jobs)
    SparkSession.builder.appName("Project 2")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config('spark.executor.memory','10g')
    .config('spark.driver.memory','12g')
    .config('spark.driver.maxResultsSize', '10 GiB')
    .config('spark.shuffle.file.buffer', '64k')
    # .config("spark.network.timeout", "3600s")
    # .master("local[6]")
    .getOrCreate()
    )

In [30]:
print("================ Mobility ================")
rrm = read_file(spark, 'Retail and Recreation.csv', type='csv', sep='|')

print("================ Cases ================")
ncd = read_file(spark, 'Net cases  day.csv', type='csv', sep='|')

================ Mobility ================
|> Loading File...
|> Loading Finished!
-RECORD 0-----------------------------------------------------------------------------------------------------
 "Date","AUS","NSW","VIC","QLD","SA","WA","TAS","NT","ACT" | "21/02/20","1","1","1","-1","2","2","-1","4","4" 
only showing top 1 row

================ Cases ================
|> Loading File...
|> Loading Finished!
-RECORD 0-----------------------------------
 "Date","Net cases / day" | "23/10/20","18" 
only showing top 1 row

